In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eVeIuFmVXuAoqDnlp9kf")
project = rf.workspace("hienlongairesearch").project("city-scapes")
version = project.version(3)
dataset = version.download("yolov8")

In [ ]:
!pip install ultralytics

In [5]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv, C2f

# Revised TGATLayer with Static Image Support
class TGATLayer(nn.Module):
    def __init__(self, channels, num_heads=4, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(channels, num_heads, dropout=dropout, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(channels, channels * 4), 
            nn.GELU(),
            nn.Linear(channels * 4, channels)
        )
        self.norm1 = nn.LayerNorm(channels)
        self.norm2 = nn.LayerNorm(channels)
        
    def forward(self, x):
        B, C, H, W = x.shape
        x_flat = x.view(B, C, -1).permute(0, 2, 1)  # [B, N, C]
        
        # Self-attention
        attn_out, _ = self.attention(x_flat, x_flat, x_flat)
        x_flat = self.norm1(x_flat + attn_out)
        
        # FFN
        ffn_out = self.ffn(x_flat)
        x_flat = self.norm2(x_flat + ffn_out)
        
        return x_flat.permute(0, 2, 1).view(B, C, H, W)

# Revised TGATNeck with Channel Management
class TGATNeck(nn.Module):
    def __init__(self, backbone_channels=[256, 512, 1024], num_heads=4):
        super().__init__()
        
        # Top-down path
        self.lateral_conv5 = Conv(backbone_channels[2], backbone_channels[1], 1)
        self.lateral_conv4 = Conv(backbone_channels[1], backbone_channels[0], 1)
        
        # TGAT blocks
        self.tgat5 = TGATLayer(backbone_channels[1], num_heads)
        self.tgat4 = TGATLayer(backbone_channels[0], num_heads)
        self.tgat3 = TGATLayer(backbone_channels[0], num_heads)
        
        # Bottom-up path
        self.up_conv3 = Conv(backbone_channels[0], backbone_channels[1], 3)
        self.up_conv4 = Conv(backbone_channels[1], backbone_channels[2], 3)

    def forward(self, features):
        c3, c4, c5 = features
        
        # Top-down
        p5 = self.lateral_conv5(c5)
        p4 = self.lateral_conv4(c4 + F.interpolate(p5, scale_factor=2))
        p3 = self.tgat3(c3 + F.interpolate(p4, scale_factor=2))
        
        # Bottom-up
        n3 = p3
        n4 = self.tgat4(p4 + self.up_conv3(F.max_pool2d(n3, 2)))
        n5 = self.tgat5(p5 + self.up_conv4(F.max_pool2d(n4, 2)))
        
        return [n3, n4, n5]

# Robust YOLOv8 Integration
class TGATYOLO(YOLO):
    def __init__(self, cfg='yolov8s.yaml', num_heads=4):
        super().__init__(cfg)
        self.replace_neck(num_heads)
        
    def replace_neck(self, num_heads):
        # Get actual backbone channels from C2f modules
        backbone_channels = [
            self.model.model[i].cv2.conv.out_channels  # Correct channel access
            for i in [4, 6, 8]  # These indices work for YOLOv8s v8.0.xx
        ]
        
        # Replace neck module (index -2 for most YOLOv8 versions)
        self.model.model[-2] = TGATNeck(backbone_channels, num_heads)

# Training Configuration
if __name__ == '__main__':
    model = TGATYOLO('yolov8s.yaml')
    model.train(
        data='/kaggle/working/City-scapes-3/data.yaml',
        epochs=50,
        batch=16,
        imgsz=640,
        lr0=0.01,
        lrf=0.1,
        optimizer="SGD",
        augment=True,
        mosaic=True,
        weight_decay=0.0005,
        dropout=0.1  # Match dropout from TGATLayer
    )

Ultralytics 8.3.85 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/kaggle/working/City-scapes-3/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

100%|██████████| 755k/755k [00:00<00:00, 10.6MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 45.2MB/s]


AMP: checks passed ✅


...


50 epochs completed in 0.704 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s summary (fused): 72 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:08<00:00,  2.39it/s]


                   all        653       5731      0.762      0.169      0.205      0.126
                ignore        490       1557      0.371    0.00643     0.0219    0.00851
            pedestrian        534       3251      0.765       0.47      0.576      0.376
        person -other-         72        104          1          0    0.00787    0.00257
          person group         79        150          1          0    0.00625    0.00146
                 rider        233        402      0.632      0.522      0.542      0.344
        sitting person         84        267      0.807      0.015     0.0768     0.0214


invalid value encountered in less
invalid value encountered in less


Speed: 0.1ms preprocess, 6.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train
